# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 27 2022 12:43PM,242864,10,MSP212601,"Methapharm, Inc.",Released
1,Jul 27 2022 12:43PM,242864,10,MSP212610,"Methapharm, Inc.",Released
2,Jul 27 2022 12:43PM,242864,10,MSP212611,"Methapharm, Inc.",Released
3,Jul 27 2022 12:22PM,242862,20,8325451,"ACI Healthcare USA, Inc.",Released
4,Jul 27 2022 12:22PM,242862,20,8325452,"ACI Healthcare USA, Inc.",Released
5,Jul 27 2022 12:22PM,242862,20,8325453,"ACI Healthcare USA, Inc.",Released
6,Jul 27 2022 12:22PM,242862,20,8325454,"ACI Healthcare USA, Inc.",Released
7,Jul 27 2022 12:22PM,242862,20,8325455,"ACI Healthcare USA, Inc.",Released
8,Jul 27 2022 12:22PM,242862,20,8325500,"ACI Healthcare USA, Inc.",Released
9,Jul 27 2022 12:22PM,242862,20,8329285,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,242859,Released,29
37,242860,Released,4
38,242861,Released,19
39,242862,Released,8
40,242864,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242859,NaN,NaN,29.0
242860,NaN,NaN,4.0
242861,NaN,NaN,19.0
242862,NaN,NaN,8.0
242864,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242781,5.0,2.0,7.0
242789,0.0,0.0,1.0
242791,0.0,0.0,1.0
242795,0.0,0.0,1.0
242798,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242781,5,2,7
242789,0,0,1
242791,0,0,1
242795,0,0,1
242798,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242781,5,2,7
1,242789,0,0,1
2,242791,0,0,1
3,242795,0,0,1
4,242798,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242781,5,2,7
1,242789,,,1
2,242791,,,1
3,242795,,,1
4,242798,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 27 2022 12:43PM,242864,10,"Methapharm, Inc."
3,Jul 27 2022 12:22PM,242862,20,"ACI Healthcare USA, Inc."
11,Jul 27 2022 12:10PM,242861,10,Bio-PRF
30,Jul 27 2022 12:08PM,242860,10,Eywa Pharma Inc.
34,Jul 27 2022 11:55AM,242859,10,ISDIN Corporation
63,Jul 27 2022 11:48AM,242858,21,"NBTY Global, Inc."
64,Jul 27 2022 11:48AM,242857,21,"NBTY Global, Inc."
65,Jul 27 2022 11:47AM,242856,21,"NBTY Global, Inc."
66,Jul 27 2022 11:47AM,242855,21,"NBTY Global, Inc."
67,Jul 27 2022 11:36AM,242852,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 27 2022 12:43PM,242864,10,"Methapharm, Inc.",,,3
1,Jul 27 2022 12:22PM,242862,20,"ACI Healthcare USA, Inc.",,,8
2,Jul 27 2022 12:10PM,242861,10,Bio-PRF,,,19
3,Jul 27 2022 12:08PM,242860,10,Eywa Pharma Inc.,,,4
4,Jul 27 2022 11:55AM,242859,10,ISDIN Corporation,,,29
5,Jul 27 2022 11:48AM,242858,21,"NBTY Global, Inc.",,,1
6,Jul 27 2022 11:48AM,242857,21,"NBTY Global, Inc.",,,1
7,Jul 27 2022 11:47AM,242856,21,"NBTY Global, Inc.",,,1
8,Jul 27 2022 11:47AM,242855,21,"NBTY Global, Inc.",,,1
9,Jul 27 2022 11:36AM,242852,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 12:43PM,242864,10,"Methapharm, Inc.",3,,
1,Jul 27 2022 12:22PM,242862,20,"ACI Healthcare USA, Inc.",8,,
2,Jul 27 2022 12:10PM,242861,10,Bio-PRF,19,,
3,Jul 27 2022 12:08PM,242860,10,Eywa Pharma Inc.,4,,
4,Jul 27 2022 11:55AM,242859,10,ISDIN Corporation,29,,
5,Jul 27 2022 11:48AM,242858,21,"NBTY Global, Inc.",1,,
6,Jul 27 2022 11:48AM,242857,21,"NBTY Global, Inc.",1,,
7,Jul 27 2022 11:47AM,242856,21,"NBTY Global, Inc.",1,,
8,Jul 27 2022 11:47AM,242855,21,"NBTY Global, Inc.",1,,
9,Jul 27 2022 11:36AM,242852,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 12:43PM,242864,10,"Methapharm, Inc.",3,,
1,Jul 27 2022 12:22PM,242862,20,"ACI Healthcare USA, Inc.",8,,
2,Jul 27 2022 12:10PM,242861,10,Bio-PRF,19,,
3,Jul 27 2022 12:08PM,242860,10,Eywa Pharma Inc.,4,,
4,Jul 27 2022 11:55AM,242859,10,ISDIN Corporation,29,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 12:43PM,242864,10,"Methapharm, Inc.",3,NaN,NaN
1,Jul 27 2022 12:22PM,242862,20,"ACI Healthcare USA, Inc.",8,NaN,NaN
2,Jul 27 2022 12:10PM,242861,10,Bio-PRF,19,NaN,NaN
3,Jul 27 2022 12:08PM,242860,10,Eywa Pharma Inc.,4,NaN,NaN
4,Jul 27 2022 11:55AM,242859,10,ISDIN Corporation,29,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 12:43PM,242864,10,"Methapharm, Inc.",3,0.0,0.0
1,Jul 27 2022 12:22PM,242862,20,"ACI Healthcare USA, Inc.",8,0.0,0.0
2,Jul 27 2022 12:10PM,242861,10,Bio-PRF,19,0.0,0.0
3,Jul 27 2022 12:08PM,242860,10,Eywa Pharma Inc.,4,0.0,0.0
4,Jul 27 2022 11:55AM,242859,10,ISDIN Corporation,29,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2914149,101,8.0,1.0
15,485643,9,20.0,0.0
19,1942469,14,11.0,4.0
20,728462,38,5.0,7.0
21,971426,4,0.0,0.0
22,485703,2,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2914149,101,8.0,1.0
1,15,485643,9,20.0,0.0
2,19,1942469,14,11.0,4.0
3,20,728462,38,5.0,7.0
4,21,971426,4,0.0,0.0
5,22,485703,2,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,101,8.0,1.0
1,15,9,20.0,0.0
2,19,14,11.0,4.0
3,20,38,5.0,7.0
4,21,4,0.0,0.0
5,22,2,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,101.0
1,15,Released,9.0
2,19,Released,14.0
3,20,Released,38.0
4,21,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21,22
Status,,,,,,
Completed,1.0,0.0,4.0,7.0,0.0,0.0
Executing,8.0,20.0,11.0,5.0,0.0,0.0
Released,101.0,9.0,14.0,38.0,4.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21,22
0,Completed,1.0,0.0,4.0,7.0,0.0,0.0
1,Executing,8.0,20.0,11.0,5.0,0.0,0.0
2,Released,101.0,9.0,14.0,38.0,4.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21,22
0,Completed,1.0,0.0,4.0,7.0,0.0,0.0
1,Executing,8.0,20.0,11.0,5.0,0.0,0.0
2,Released,101.0,9.0,14.0,38.0,4.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()